In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
from keras import datasets, layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, Rescaling, Conv2D, MaxPooling2D, Flatten


In [34]:
# Create a training dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
   'C:\\Users\\dell\\Documents\\skin classification\\Skin_Conditions',
    image_size=(224, 224),  # Resize images to the input size of the model
    batch_size=32,          # Number of images in each batch
    label_mode='categorical',  # 'categorical' for multi-class classification
    seed=123,               # Seed for shuffling the dataset
    validation_split=0.2,   # Reserves 20% of the dataset for validation
    subset='training')     # Use this part of the dataset for training

Found 2394 files belonging to 6 classes.
Using 1916 files for training.


In [35]:
# Create a validation dataset from the remaining 20% of the data
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'C:\\Users\\dell\\Documents\\skin classification\\Skin_Conditions',
    image_size=(224, 224),
    batch_size=32,
    label_mode='categorical',
    seed=123,
    validation_split=0.2,
    subset='validation'
)

Found 2394 files belonging to 6 classes.
Using 478 files for validation.


In [ ]:
#To see the labels for the images
for images, labels in train_dataset:
    print(labels.numpy())#Converts from Tensor to array

In [ ]:
#To see the images
for images, labels in train_dataset.take(20):  # Takes the first batch
    # Select an image from the batch
    image = images[15].numpy().astype("uint8")  # Convert tensor to NumPy array so it can be plotted

plt.figure()
plt.imshow(image)
plt.colorbar()
plt.show()

In [23]:
'''You could add a normalization layer to both the training and validation dataset to keep gradients
in a manaeageable range and for numerical stability 
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
validation_dataset = validation_dataset.map(lambda x, y: (normalization_layer(x), y))'''

In [ ]:
#Build the CNN model
#Convolutional Base
model = models.Sequential(
    [#Add Preprocessing layers like data augmentaion and normalization
    RandomFlip("horizontal",input_shape=(224, 224, 3)),
    RandomRotation(0.1),
    RandomZoom(0.1),
    Rescaling(1./255)
    ]
)
model.add(layers.Conv2D(32,(3,3), activation='relu',input_shape=(224,224,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3), activation='relu'))

#Adding Dense Layers
model.add(layers.Flatten())
model.add(layers.Dense(64,activation='relu'))
model.add(layers.Dense(6,activation='softmax'))

#Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',  # Use this for categorical labels
              metrics=['accuracy'])


c:\Users\dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip_4 (RandomFlip)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_rotation_4               │ (None, 224, 224, 3)    │             0 │
│ (RandomRotation)                │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ random_zoom_4 (RandomZoom)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_6 (Rescaling)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 52, 52, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_6 (Flatten)             │ (None, 173056)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 64)             │    11,075,648 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 6)              │           390 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,132,358 (42.47 MB)

 Trainable params: 11,132,358 (42.47 MB)

 Non-trainable params: 0 (0.00 B)

In [42]:
#To train the model on the data
model.fit(train_dataset, validation_data=validation_dataset, epochs=8)

Epoch 1/8
60/60 ━━━━━━━━━━━━━━━━━━━━ 84s 1s/step - accuracy: 0.2324 - loss: 2.2003 - val_accuracy: 0.3745 - val_loss: 1.4840
Epoch 2/8
60/60 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.4362 - loss: 1.4492 - val_accuracy: 0.4812 - val_loss: 1.3103
Epoch 3/8
60/60 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.5066 - loss: 1.3599 - val_accuracy: 0.5063 - val_loss: 1.2411
Epoch 4/8
60/60 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - accuracy: 0.5302 - loss: 1.2801 - val_accuracy: 0.5167 - val_loss: 1.2158
Epoch 5/8
60/60 ━━━━━━━━━━━━━━━━━━━━ 80s 1s/step - accuracy: 0.5393 - loss: 1.1694 - val_accuracy: 0.4958 - val_loss: 1.2442
Epoch 6/8
60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.5582 - loss: 1.1143 - val_accuracy: 0.5586 - val_loss: 1.0862
Epoch 7/8
60/60 ━━━━━━━━━━━━━━━━━━━━ 93s 2s/step - accuracy: 0.6044 - loss: 1.0443 - val_accuracy: 0.5837 - val_loss: 1.1185
Epoch 8/8
60/60 ━━━━━━━━━━━━━━━━━━━━ 98s 2s/step - accuracy: 0.6280 - loss: 0.9649 - val_accuracy: 0.6255 - val_loss: 1.0649


In [47]:
#Add new dense layers to further increase accuracy by understanding more features
from tensorflow.keras.layers import Dense, Dropout

_ = model(tf.keras.Input(shape=model.input_shape[1:]))

base_output=model.output
x=Dense(128,activation='relu')(base_output)#(base_output) appplies the layer to the previous version of the model
x=Dense(64, activation='relu')(x)
x=Dense(32, activation='relu')(x)
final_output=Dense(6, activation='softmax')(x)

new_model=Model(inputs=model.input,outputs=final_output)
new_model.compile(optimizer='adam',
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [48]:
#Train the new model
new_model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.3202 - loss: 0.0000e+00 - val_accuracy: 0.3661 - val_loss: 0.0000e+00
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.3454 - loss: 0.0000e+00 - val_accuracy: 0.4079 - val_loss: 0.0000e+00
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.4005 - loss: 0.0000e+00 - val_accuracy: 0.3222 - val_loss: 0.0000e+00
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.3244 - loss: 0.0000e+00 - val_accuracy: 0.3870 - val_loss: 0.0000e+00
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.4098 - loss: 0.0000e+00 - val_accuracy: 0.3431 - val_loss: 0.0000e+00
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 68s 1s/step - accuracy: 0.4062 - loss: 0.0000e+00 - val_accuracy: 0.4289 - val_loss: 0.0000e+00
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.4627 - loss: 0.0000e+00 - val_accuracy: 0.4498 - val_loss: 0.0000e+00
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 

In [49]:
#Train on more epochs to improve accuracy
new_model.fit(train_dataset, validation_data=validation_dataset, initial_epoch=10, epochs=15,)

Epoch 11/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.5749 - loss: 0.0000e+00 - val_accuracy: 0.5042 - val_loss: 0.0000e+00
Epoch 12/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.5613 - loss: 0.0000e+00 - val_accuracy: 0.5272 - val_loss: 0.0000e+00
Epoch 13/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - accuracy: 0.6005 - loss: 0.0000e+00 - val_accuracy: 0.5753 - val_loss: 0.0000e+00
Epoch 14/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 62s 1s/step - accuracy: 0.6268 - loss: 0.0000e+00 - val_accuracy: 0.5628 - val_loss: 0.0000e+00
Epoch 15/15
60/60 ━━━━━━━━━━━━━━━━━━━━ 64s 1s/step - accuracy: 0.6451 - loss: 0.0000e+00 - val_accuracy: 0.5460 - val_loss: 0.0000e+00


In [50]:
new_model.fit(train_dataset, validation_data=validation_dataset, initial_epoch=15, epochs=30,)

Epoch 16/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.6145 - loss: 0.0000e+00 - val_accuracy: 0.6088 - val_loss: 0.0000e+00
Epoch 17/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - accuracy: 0.6790 - loss: 0.0000e+00 - val_accuracy: 0.5816 - val_loss: 0.0000e+00
Epoch 18/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.6805 - loss: 0.0000e+00 - val_accuracy: 0.6046 - val_loss: 0.0000e+00
Epoch 19/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.6892 - loss: 0.0000e+00 - val_accuracy: 0.5921 - val_loss: 0.0000e+00
Epoch 20/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 60s 1s/step - accuracy: 0.7027 - loss: 0.0000e+00 - val_accuracy: 0.6297 - val_loss: 0.0000e+00
Epoch 21/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.7094 - loss: 0.0000e+00 - val_accuracy: 0.5921 - val_loss: 0.0000e+00
Epoch 22/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.6837 - loss: 0.0000e+00 - val_accuracy: 0.5962 - val_loss: 0.0000e+00
Epoch 23/30
60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - ac

In [54]:
#The model is beginning to overfit data, while still lacking in accuracy
#Add the dropout function to reduce overfitting
#Use another activation function to see if the model improves
from tensorflow.keras.layers import Dropout, PReLU

base_output = new_model.output
x = Dense(128)(base_output)
x = PReLU()(x)
x = Dropout(0.5)(x)  # Drop 50% of neurons to prevent overfitting

x = Dense(64)(x)
x = PReLU()(x)

x = Dense(64)(x)
x = PReLU()(x)

final_output = Dense(6, activation='softmax')(x)

updated_model = Model(inputs=model.input, outputs=final_output)

updated_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [55]:
updated_model.fit(train_dataset, validation_data=validation_dataset, epochs=10)

Epoch 1/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 85s 1s/step - accuracy: 0.3365 - loss: 0.0000e+00 - val_accuracy: 0.5628 - val_loss: 0.0000e+00
Epoch 2/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.6054 - loss: 0.0000e+00 - val_accuracy: 0.6318 - val_loss: 0.0000e+00
Epoch 3/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.6934 - loss: 0.0000e+00 - val_accuracy: 0.6381 - val_loss: 0.0000e+00
Epoch 4/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.6634 - loss: 0.0000e+00 - val_accuracy: 0.6527 - val_loss: 0.0000e+00
Epoch 5/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.6919 - loss: 0.0000e+00 - val_accuracy: 0.6715 - val_loss: 0.0000e+00
Epoch 6/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 0.7259 - loss: 0.0000e+00 - val_accuracy: 0.6464 - val_loss: 0.0000e+00
Epoch 7/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.7243 - loss: 0.0000e+00 - val_accuracy: 0.6757 - val_loss: 0.0000e+00
Epoch 8/10
60/60 ━━━━━━━━━━━━━━━━━━━━ 74s 1s/step - accuracy: 

In [56]:
updated_model.fit(train_dataset, validation_data=validation_dataset, initial_epoch=10, epochs=17,)

Epoch 11/17
60/60 ━━━━━━━━━━━━━━━━━━━━ 77s 1s/step - accuracy: 0.7467 - loss: 0.0000e+00 - val_accuracy: 0.6757 - val_loss: 0.0000e+00
Epoch 12/17
60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.7457 - loss: 0.0000e+00 - val_accuracy: 0.6674 - val_loss: 0.0000e+00
Epoch 13/17
60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.7714 - loss: 0.0000e+00 - val_accuracy: 0.6841 - val_loss: 0.0000e+00
Epoch 14/17
60/60 ━━━━━━━━━━━━━━━━━━━━ 76s 1s/step - accuracy: 0.7575 - loss: 0.0000e+00 - val_accuracy: 0.6778 - val_loss: 0.0000e+00
Epoch 15/17
60/60 ━━━━━━━━━━━━━━━━━━━━ 78s 1s/step - accuracy: 0.7911 - loss: 0.0000e+00 - val_accuracy: 0.6946 - val_loss: 0.0000e+00
Epoch 16/17
60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.7724 - loss: 0.0000e+00 - val_accuracy: 0.6904 - val_loss: 0.0000e+00
Epoch 17/17
60/60 ━━━━━━━━━━━━━━━━━━━━ 75s 1s/step - accuracy: 0.7864 - loss: 0.0000e+00 - val_accuracy: 0.7008 - val_loss: 0.0000e+00


In [58]:
test_loss, test_acc=model.evaluate(validation_dataset,verbose=2) #Verbose=2 setting means that during model training, the system will show one line of output per epoch, displaying key training metrics such as loss and accuracy but omitting detailed information about each batch.
print(test_acc)

15/15 - 5s - 302ms/step - accuracy: 0.6883 - loss: 1.2388
0.6882845163345337


In [57]:
updated_model.save('skin_condition_classifier.keras')